# The While language

The While language is a procedural language that is commonly used for learning program analysis. Is the barebones of an imperative language that can be extended for reasoning about other properties of programs.

The abstract syntax of the While language is as follows

In [1]:
type var = string
type label = int

type expr = Var of var 
          | Const of int
          | Op of string * expr * expr
type bool_expr = True | False
               | Not of bool_expr
               | BoolOp of string * bool_expr * bool_expr
               | RelOp of string * expr * expr
type stmt = Skip of label
          | Assign of var * expr * label
          | Seq of stmt * stmt
          | If of bool_expr * label * stmt * stmt
          | While of bool_expr * label * stmt

type var = string


type label = int


type expr = Var of var | Const of int | Op of string * expr * expr


type bool_expr =
    True
  | False
  | Not of bool_expr
  | BoolOp of string * bool_expr * bool_expr
  | RelOp of string * expr * expr


type stmt =
    Skip of label
  | Assign of var * expr * label
  | Seq of stmt * stmt
  | If of bool_expr * label * stmt * stmt
  | While of bool_expr * label * stmt


We can define an example program with this syntax:

```python
y = x          # 1
z = 1          # 2
while y > 1:   # 3
    z = z * y  # 4
    y = y - 1  # 5
y = 0          # 6
```

Each number represent the label of that part of the program. In OCaml syntax this program will look like this

In [25]:
module Examples = struct
    let factorial = Seq (
                            Assign ("y", (Var "x"), 1),
                            Seq (
                                    Assign ("z", (Const 1), 2),
                                    Seq (
                                            While (
                                                    RelOp (">", (Var "y"), (Const 1)), 
                                                    3, 
                                                    Seq (
                                                            Assign ("z", Op ("*", (Var "z"), (Var "y")), 4),
                                                            Assign ("y", Op ("-", (Var "y"), (Const 1)), 5)
                                                        )
                                                  ),
                                            Assign ("y", (Const 0), 6)
                                        )
                                )
                        )
end

module Examples : sig val factorial : stmt end


For the analysis of the programs we will need mostly three things from the program:

- The variables in the program
- The labels defined in the program
- The call graph

We can write some utility functions that will give us this information

In [52]:
module VarsSet = Set.Make(String)

let rec vars_of_expr = function
| Var v -> VarsSet.singleton v
| Const _ -> VarsSet.empty
| Op (_, e1, e2) -> VarsSet.union (vars_of_expr e1) (vars_of_expr e2)

let rec vars_of_bool_expr = function
| True | False -> VarsSet.empty
| Not b -> vars_of_bool_expr b
| BoolOp (_, b1, b2) -> VarsSet.union (vars_of_bool_expr b1) (vars_of_bool_expr b2)
| RelOp (_, e1, e2) -> VarsSet.union (vars_of_expr e1) (vars_of_expr e2)

let rec vars_of_stmt = function
| Skip _ -> VarsSet.empty
| Assign (v, e, _) -> VarsSet.union (VarsSet.singleton v) (vars_of_expr e)
| Seq (s1, s2) -> VarsSet.union (vars_of_stmt s1) (vars_of_stmt s2)
| If (b, _, s1, s2) -> VarsSet.union (VarsSet.union (vars_of_bool_expr b) (vars_of_stmt s1)) (vars_of_stmt s2)
| While (b, _, s) -> VarsSet.union (vars_of_bool_expr b) (vars_of_stmt s)

module LabelsSet = Set.Make(struct 
   type t = int 
   let compare = compare
end)
 
let rec labels_of_stmt = function
| Skip l -> LabelsSet.singleton l
| Assign (_, _, l) -> LabelsSet.singleton l
| Seq (s1, s2) -> LabelsSet.union (labels_of_stmt s1) (labels_of_stmt s2)
| If (_, l, s1, s2) -> LabelsSet.union (LabelsSet.union (LabelsSet.singleton l) (labels_of_stmt s1)) (labels_of_stmt s2)
| While (_, l, s) -> LabelsSet.union (LabelsSet.singleton l) (labels_of_stmt s)

type graph_node = Begin | End
                | Assign of var * expr * label
                | BoolExpr of bool_expr * label
type graph_edge = graph_node * graph_node

module GraphNodesSet = Set.Make(struct
    type t = graph_node
    let compare = compare
end)

module GraphEdgesSet = Set.Make(struct
    type t = graph_edge
    let compare = compare
end)

type cfg = GraphNodesSet.t * GraphEdgesSet.t * graph_node

let empty_cfg = (GraphNodesSet.of_list [Begin; End]), (GraphEdgesSet.singleton (Begin, End)), Begin

let build_cfg cfg stmt = "TODO"

type program_info = VarsSet.t * LabelsSet.t * cfg

module VarsSet :
  sig
    type elt = String.t
    type t = Set.Make(String).t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : 

val vars_of_expr : expr -> VarsSet.t = <fun>


val vars_of_bool_expr : bool_expr -> VarsSet.t = <fun>


val vars_of_stmt : stmt -> VarsSet.t = <fun>


module LabelsSet :
  sig
    type elt = int
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : elt -> t -> t * bool * t

...

module GraphEdgesSet :
  sig
    type elt = graph_edge
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : elt -> t -> t

type cfg = GraphNodesSet.t * GraphEdgesSet.t * graph_node


val empty_cfg : GraphNodesSet.t * GraphEdgesSet.t * graph_node =
  (<abstr>, <abstr>, Begin)


val build_cfg : 'a -> 'b -> string = <fun>


type program_info = VarsSet.t * LabelsSet.t * cfg
